In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import re


In [2]:
def emotion2sentiment(emotion):
    if emotion == 'angry':
        return 0
    elif emotion == 'fear':
        return 1
    elif emotion == 'sad':
        return 2
    elif emotion == 'neural':
        return 3
    elif emotion == 'surprise':
        return 4
    elif emotion == 'happy':
        return 5
    
def sentiment2emotion(sentiment):
    if sentiment == 0:
        return "angry"
    elif sentiment == 1:
        return "fear"
    elif sentiment == 2:
        return "sad"
    elif sentiment == 3:
        return "neural"
    elif sentiment == 4:
        return "surprise"
    elif sentiment == 5:
        return "happy"

In [3]:
def remove_symbol(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line

In [4]:
import jieba

def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [5]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

stop_words_file = '哈工大停用词表.txt'
stopwords = get_custom_stopwords(stop_words_file)
# # print(type(stopwords))
# vect = CountVectorizer(max_df = 0.8, 
#                        min_df = 3, 
#                        token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b', 
#                        stop_words=stopwords)
# vect = TfidfVectorizer(use_idf=True, smooth_idf=True,max_df = 0.8, 
#                        min_df = 3, 
#                        token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b', 
#                        stop_words=stopwords)

In [6]:

# 设置最频繁使用的50000个词
MAX_NB_WORDS = 50000
# 每条cut_review最大的长度
MAX_SEQUENCE_LENGTH = 250
# 设置Embeddingceng层的维度
EMBEDDING_DIM = 100
 
# tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
# tokenizer.fit_on_texts(data_train['cut_text'].values)
# word_index = tokenizer.word_index
# print('共有 %s 个不相同的词语.' % len(word_index))
def load_tokenizer():
    f = open('tokenizer_json.json', 'r')
    content = f.read()
#     print(type(content))
    f.close()
    _tokenizer = keras.preprocessing.text.tokenizer_from_json(content)
    return _tokenizer

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def load_model():
    model = keras.models.load_model("LSTM_model_20230301")
    return model

In [8]:
def my_predict(test_input):
    # test_input = input("input the sentence:")
#     test_input = "今天真开心"
    test_cut = chinese_word_cut(remove_symbol(test_input))
    # print(type(test_cut))
    test_array = np.array([test_cut])
    test_X = my_tokenizer.texts_to_sequences(test_array)
    # print(len(test_X))
    test_X = pad_sequences(test_X, maxlen=MAX_SEQUENCE_LENGTH)
    # print(test_X.shape)
    result_array = my_model.predict(test_X).argmax(axis=1)
    # print(result_array)
    result_emotion = sentiment2emotion(result_array[0])
    return result_emotion
    # print(result_emotion)
    # result_sentiment = np.argmax(np.bincount(result_array))
    # result_emotion = sentiment2emotion(result_sentiment)
    # # result = reconstructed_model.predict(test_X)
    # # ticklabels = pd.Series(range(len(data_test.emotion.unique()))).apply(sentiment2emotion)
    # print(result_emotion)
    # # print(type(result))
    # # print(test_X.shape)

In [9]:
def init():
    global my_tokenizer
    global my_model
    my_tokenizer = load_tokenizer()
    my_model = load_model()
    
    

In [10]:
# init()
# # text = input("Please input the text: ")
# res = my_predict("生气")
# print("The emotion of the text is " + res)

In [11]:
import PySimpleGUI as sg

In [12]:
init()
sg.theme('GrayGrayGray')   # 设置当前主题
# 界面布局，将会按照列表顺序从上往下依次排列，二级列表中，从左往右依此排列
layout = [  [sg.Text('Please input the text: '), sg.InputText(key='-IN-')],
            [sg.Text(key='-OUTPUT-')],
            [sg.Button('Ok'), sg.Button('Cancel')] ]

# 创造窗口
window = sg.Window('Automatic Sentiment Analysis of Text', layout)
# 事件循环并获取输入值
while True:
    event, values = window.read()
    if event in (None, 'Cancel'):   # 如果用户关闭窗口或点击`Cancel`
        break
    if event == 'Ok':
        res = my_predict(values['-IN-'])
        window['-OUTPUT-'].update("The sentiment of the input text is " + res)
    

window.close()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Gao\AppData\Local\Temp\jieba.cache
Loading model cost 0.480 seconds.
Prefix dict has been built successfully.


1/1 [==============================] - 0s 24ms/step


In [13]:
# sg.theme_previewer()